In [ ]:
# |default_exp collector

In [ ]:
# |export
import asyncio
import json
import logging
import traceback
from copy import copy, deepcopy
from functools import cached_property
from itertools import cycle, repeat
from pathlib import Path
from queue import Full
from time import sleep, time

import numpy as np
import pandas as pd
import psutil
import torch
import wandb
from fastapi import FastAPI, Request
from fastcore.basics import cycle, store_attr
from fastcore.foundation import L, Self
from guppy import hpy
from tmenv.gbx import map_from_uid
from tmlearn.callback import Callback, Callbackable
from tmlearn.net import ThreadServer
from torch.nn.functional import relu
from tmlearn.utils import Timer, format_date
from torch.multiprocessing import Process
from torchvision.transforms import Resize

# Collector
> Collect experiences and put them to a replay buffer

In [ ]:
# |export
class CollectorWorker(Process):
    def __init__(
        self,
        env,
        cbs,
        agent,
        replay_buffer,
        session_scheduler,
        next_map_queue,
        done_map_queue,
    ):
        assert env.device == agent.device
        super().__init__()
        store_attr()


    def run(self):
        with torch.cuda.stream(torch.cuda.Stream(self.env.device, -1)), torch.no_grad():
            try:
                self.session_collector = SessionCollector(
                    self.cbs, self.env, self.agent, self.replay_buffer, self.session_scheduler
                )
                self.env.game  # Start the game
                while (map_uid := self.next_map_queue.get()) is not None:
                    self.session_collector.execute(map_uid)
                    self.done_map_queue.put(map_uid)
            except KeyboardInterrupt:
                logging.info("Received KeyboardInterrupt, destroying env and worker !")
                self.env.close()


class SessionCollector(Callbackable):
    def __init__(self, cbs, env, agent, replay_buffer, session_scheduler):
        super().__init__(cbs)
        store_attr(but="cbs")
        self("after_init")

    def execute(self, map_uid):
        self.map_uid = map_uid
        self.session_scheduler.reset()
        self("before_load_map")
        loaded = False
        while not loaded:
            try:
                self.env.load_map(self.map_uid)
                loaded = True
            except Exception as e:
                logging.error(traceback.format_exc())
                self.env.recover()
        self("before_session")
        while not self.session_scheduler.done:
            try:
                self.collect_episode()
            except Exception as e:
                logging.error(traceback.format_exc())
                self.env.recover()
            except CancelEpisodeException:
                pass
        self.session_scheduler.end_session()
        self("after_session")

    def collect_episode(self):
        self.state = self.env.reset()
        self.info = dict()
        self("after_reset")
        while not self.collect_step():
            pass
        self.env.game.act(False, False, 0)
        self.session_scheduler.end_episode(self)
        # <<< max_progress hack: quick and dirty but project is at its end of life
        self.info["progress"] = self.info["max_progress"]
        del self.info["max_progress"]
        # >>> max_progress hack: end
        self("after_done")

    def collect_step(self):
        self("before_collect_step")
        self.action = self.agent.select_action(self.state)
        self("before_env_step")
        self.next_state, self.reward, self.done, self.info = self.env.step(self.action)
        try:
            self("after_env_step")
            self.replay_buffer.push(
                self.state,
                self.next_state,
                self.action,
                self.reward,
                self.done or self.info["must_respawn"],
            )
        except CancelPushException:
            pass
        self("after_step_push")
        if self.info["must_respawn"]:
            self("before_respawn")
            self.state = self.env.respawn(self.info["respawn_type"] == "launched")
        else:
            self.state = self.next_state
        self("after_collect_step")
        return self.done

    def state_dict(self):
        return dict(
            session_scheduler=self.session_scheduler.state_dict(),
            cbs=self.cbs_state_dict(),
        )

    def load_state_dict(self, state_dict):
        self.session_scheduler.load_state_dict(state_dict["session_scheduler"])
        if "cbs" in state_dict:
            self.load_cbs_state_dict(state_dict["cbs"])

# Session scheduler

In [ ]:
# |export
class SessionScheduler:
    def __init__(self, nb_episode):
        store_attr()

    @property
    def done(self):
        return self.nb_episode <= self.round

    def reset(self):
        self.session = 0
        self.round = 0

    def end_episode(self, collector):
        collector.info["session"] = self.session
        self.round += 1

    def end_session(self):
        self.session += 1
        self.round = 0

    def state_dict(self):
        return dict(
            round=self.round,
            session=self.session,
            nb_episode=self.nb_episode,
        )

    def load_state_dict(self, state_dict):
        self.round = state_dict["round"]
        self.session = state_dict["session"]

## Exceptions

In [ ]:
# |export
class CancelEpisodeException(Exception):
    pass


class RecoverEnvException(Exception):
    pass


class CancelPushException(Exception):
    pass

## Callbacks

In [ ]:
# |export
class WandbCollectorCallback(Callback):
    def __init__(
        self,
        project,
        entity,
        group,
    ):
        store_attr()

    @cached_property
    def wandb_api(self):
        return wandb.Api()

    def load_run(self, run):
        assert len(run) == 1, "Several runs found"
        self.run = wandb.init(
            project=self.project,
            entity=self.entity,
            id=run[0].id,
            resume="must",
            config=self.parent.env.config,
            reinit=True,
        )
        state_dict = self.run.summary.get("state_dict")
        if state_dict:
            self.parent.load_state_dict(json.loads(state_dict))

    def create_run(self):
        name = self.parent.env.map_data["sanitized_name"]
        self.run = wandb.init(
            project=self.project,
            entity=self.entity,
            name=name,
            tags=[name, "collect"],
            job_type="collect",
            group=self.group,
            config=self.parent.env.config,
            reinit=True,
        )

    def before_session(self):
        self.load_run(run) if (run := self.existing_run) else self.create_run()
        self.run.summary["state_dict"] = json.dumps(self.parent.state_dict())
        self.ghost_saver = WandbGhostSaver(self.run)

    @property
    def wandb_path(self):
        return f"{self.entity}/{self.project}"

    @property
    def existing_run(self):
        filters = {
            "group": self.group,
            "config.map.uid": self.parent.env.map_data["uid"],
        }
        return list(self.wandb_api.runs(self.wandb_path, filters))

    def after_done(self):
        self.parent.env.game.save_ghost(self.ghost_saver.save_url, str(self.run.step))
        self.run.log(self.parent.info)
        self.run.summary["state_dict"] = json.dumps(self.parent.state_dict())

    def after_session(self):
        self.run.summary["state_dict"] = json.dumps(self.parent.state_dict())
        self.run.finish()
        del self.ghost_saver
        del self.run
        self.wandb_api.flush()


class WandbGhostSaver(ThreadServer):
    def __init__(self, run, dir_size=1000):
        store_attr()
        self.base_dir = Path(self.run.dir) / "ghosts"
        self.base_dir.mkdir(parents=True, exist_ok=True)
        self.run.save("ghosts/*", policy="now")

        async def save(episode, ghost):
            episode = int(episode)
            dir_name = str(episode - (episode % self.dir_size))
            save_dir = self.base_dir / dir_name
            save_dir.mkdir(parents=True, exist_ok=True)
            filepath = save_dir / f"ghost_{episode}.gbx"
            with filepath.open("wb") as f:
                f.write(ghost)
            self.run.save("ghosts/*", policy="now")

        app = FastAPI()

        @app.post("/save")
        async def app_save(request: Request):
            asyncio.get_event_loop().create_task(
                save(request.headers["episode_id"], await request.body())
            )
            return {}

        super().__init__(app, host="127.0.0.1", port=0, log_level="error")

    @property
    def save_url(self):
        return f"http://127.0.0.1:{self.port}/save"

In [ ]:
# |export
class ExplorationSwitcherCallback(Callback):
    def __init__(self, explorations):
        store_attr()

    def before_session(self):
        self.cycle = cycle(self.explorations)
        self.parent.agent.exploration = next(self.cycle)
        self.nb_switch = 1

    def after_done(self):
        self.parent.agent.exploration = next(self.cycle)
        self.nb_switch += 1

    def after_session(self):
        self.cycle = cycle(self.explorations)
        self.parent.agent.exploration = next(self.cycle)
        self.nb_switch = 1

    def state_dict(self):
        return dict(nb_switch=self.nb_switch)

    def load_state_dict(self, state_dict):
        self.cycle = cycle(self.explorations)
        self.nb_switch = state_dict["nb_switch"]
        for _ in range(self.nb_switch):
            self.parent.agent.exploration = next(self.cycle)

In [ ]:
# |export
class TelemetryTimeoutRecoverCallback(Callback):
    def __init__(self, timeout=10.0):
        store_attr()

    def after_env_step(self):
        delay = time() - self.parent.env.game.telemetry["_updatetime"]
        if self.timeout < delay:
            raise RecoverEnvException(
                f"Telemetry delay {delay} > timeout {self.timeout}"
            )


class MemoryFullRecoverCallback(Callback):
    def __init__(self, percent_threshold=90.0):
        store_attr()

    def after_reset(self):
        percent = psutil.virtual_memory().percent
        if self.percent_threshold < percent:
            raise RecoverEnvException(
                f"System memory usage {percent}% > threshold {self.percent_threshold}%"
            )


class IgnoreLaggyStepCallback(Callback):
    def __init__(self, timeout=0.1):
        store_attr()

    def after_env_step(self):
        if self.timeout < time() - self.parent.env.game.telemetry["_updatetime"]:
            raise CancelPushException()


class IgnoreRespawnAndUnfinished(Callback):
    def after_env_step(self):
        if self.parent.info["must_respawn"] or (
            self.parent.done and not self.parent.info["finished"]
        ):
            raise CancelPushException()


class IgnoreFinished(Callback):
    def after_env_step(self):
        if self.parent.info["finished"]:
            raise CancelPushException()

In [ ]:
# |export
class LagCallback(Callback):
    def before_collect_step(self):
        self.bcs_time = time()

    def before_env_step(self):
        self.bes_time = time()

    def after_env_step(self):
        self.aes_time = time()

    def after_collect_step(self):
        acs_time, ts = time(), self.parent.env.timestep
        self.parent.info["collect_lag"] = (acs_time - self.bcs_time - ts) * 1000
        self.parent.info["select_lag"] = (self.bes_time - self.bcs_time) * 1000
        self.parent.info["env_lag"] = (self.aes_time - self.bes_time - ts) * 1000
        self.parent.info["push_lag"] = (acs_time - self.aes_time) * 1000

In [ ]:
# |export
def publish(queue, payload):
    try:
        queue.put_nowait(payload)
    except Full:
        pass


class PublisherCallback(Callback):
    def __init__(self, publisher_queue):
        store_attr()

    def publish(self, payload):
        publish(self.publisher_queue, payload)

    def publish_kwargs(self, **kwargs):
        publish(self.publisher_queue, kwargs)


class ActionPublisherCallback(PublisherCallback):
    def before_env_step(self):
        data = self.parent.env.action_handler.action_dict(self.parent.action)
        if hasattr(self.parent.agent, "exploration"):
            data |= self.parent.agent.exploration.state_dict()
        self.publish(data)

    def before_respawn(self):
        data = self.parent.env.action_handler.default_action_dict
        if hasattr(self.parent.agent, "exploration"):
            data |= self.parent.agent.exploration.state_dict()
        self.publish(data)


class StepPublisherCallback(PublisherCallback):
    def after_step_push(self):
        data = self.parent.next_state["env"]
        data |= dict(reward=self.parent.reward)
        data |= self.parent.info
        data["done"] = self.parent.done
        self.publish(data)

In [ ]:
#|export
class EigenCamPublisherCallback(PublisherCallback):
    
    def after_init(self, output_size=(36, 64)):
        self.resize = Resize(output_size, antialias=True)
        self.parent.agent.net_online.model.visual.raw_model.final_conv.register_forward_hook(self.save_activations)

    def save_activations(self, module, input, output):
        self.activations = output.detach().squeeze(0)

    def before_env_step(self):
        cam = self.activations.reshape(self.activations.shape[0], -1).t()
        cam = cam - cam.mean(axis=0)
        cam = cam @ cam.svd().V[:, 0]
        cam = relu(cam.reshape(self.activations.shape[1:]))
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-7)
        cam = self.resize(cam.unsqueeze(0)).squeeze()
        self.publish(cam.tolist())

In [ ]:
#|export
class SessionStats(dict):
    @classmethod
    def from_df(cls, df):
        return cls(
            finishes=cls.empty_finishes() | df["medal"].value_counts().to_dict(),
            round_count=len(df),
            best=dict(
                progress=float(df["progress"].max()),
                total_reward=float(df["total_reward"].max()),
                finish_racetime=cls.safe_min_from_df(df, "finish_racetime"),
                no_respawn_reward=float(df["no_respawn_reward"].max()),
                no_respawn_finish_racetime=cls.safe_min_from_df(
                    df, "no_respawn_finish_racetime"
                ),
            ),
        )

    @staticmethod
    def safe_min_from_df(df, column, _type=int, default=None):
        if column in df and df[column].notnull().any():
            return _type(df[column].min())
        return default

    @classmethod
    def empty(cls):
        return cls(
            finishes=cls.empty_finishes(),
            round_count=0,
            best=dict(
                progress=0,
                total_reward=None,
                finish_racetime=None,
                no_respawn_reward=None,
                no_respawn_finish_racetime=None,
            ),
        )

    @staticmethod
    def empty_finishes():
        return dict(
            unfinished=0,
            no_medal=0,
            bronze=0,
            silver=0,
            gold=0,
            author=0,
        )

    def update_after_episode(self, info):
        self["finishes"][info["medal"]] += 1
        self["round_count"] += 1
        self.update_best_value(max, "progress", info, 0)
        self.update_best_value(max, "total_reward", info)
        self.update_best_value(min, "finish_racetime", info)
        self.update_best_value(max, "no_respawn_reward", info)
        self.update_best_value(min, "no_respawn_finish_racetime", info)

    def update_best_value(self, fn, key, info, default=None):
        self["best"][key] = fn(
            L(self["best"], info)
            .filter(lambda o: key in o and o[key] != default)
            .map(Self[key]),
            default=default,
        )


class SessionPublisherCallback(PublisherCallback):
    """Require WandbCollectorCallback"""

    def before_load_map(self):
        self.publish_kwargs(
            status="before_load_map",
            map=map_from_uid(self.parent.map_uid),
        )

    def before_session(self):
        df = pd.DataFrame(
            self.parent.wandb_collector.wandb_api.run(
                self.parent.wandb_collector.run.path
            ).scan_history(page_size=10**9)
        )
        if 0 < len(df):
            self.stats = dict(
                overall=SessionStats.from_df(df),
                sessions={
                    group[0]: SessionStats.from_df(group[1])
                    for group in df.groupby("session")
                },
            )
            if self.parent.session_scheduler.session not in self.stats["sessions"]:
                self.stats["sessions"][
                    self.parent.session_scheduler.session
                ] = SessionStats.empty()
        else:
            self.stats = dict(
                overall=SessionStats.empty(),
                sessions={self.parent.session_scheduler.session: SessionStats.empty()},
            )
        self.stats["current_session"] = self.parent.session_scheduler.session
        self.publish_kwargs(
            status="before_session",
            stats=self.stats,
            config=self.parent.env.config,
            state=self.parent.state_dict(),
        )

    def after_reset(self):
        self.publish_kwargs(
            status="after_reset",
            stats=self.stats,
            config=self.parent.env.config,
            state=self.parent.state_dict(),
        )

    def after_done(self):
        previous_overall_best = copy(self.stats["overall"]["best"])
        previous_session_best = copy(
            self.stats["sessions"][self.parent.info["session"]]["best"]
        )
        self.stats["overall"].update_after_episode(self.parent.info)
        self.stats["sessions"][self.parent.info["session"]].update_after_episode(
            self.parent.info
        )
        self.publish_kwargs(
            status="after_done",
            stats=self.stats,
            config=self.parent.env.config,
            state=self.parent.state_dict(),
            info=self.parent.info,
            previous_overall_best=previous_overall_best,
            previous_session_best=previous_session_best,
        )

    def after_session(self):
        self.publish_kwargs(
            status="after_session",
            stats=self.stats,
            config=self.parent.env.config,
            state=self.parent.state_dict(),
        )
        sleep(1)

In [ ]:
# |hide
# |eval: false
from nbdev.doclinks import nbdev_export

nbdev_export()